In [ ]:
import numpy as np
from categorical_scatter import categorical_scatter_2d

class TSNE():

    def neg_squared_euc_dists(X):
        """Compute matrix containing negative squared euclidean
        distance for all pairs of points in input matrix X
        # Arguments:
            X: matrix of size NxD
        # Returns:
            NxN matrix D, with entry D_ij = negative squared
            euclidean distance between rows X_i and X_j
        """
        # Math? See https://stackoverflow.com/questions/37009647
        sum_X = np.sum(np.square(X), 1)
        D = np.add(np.add(-2 * np.dot(X, X.T), sum_X).T, sum_X)
        return -D


    def softmax(X, diag_zero=True, zero_index=None):
        """Compute softmax values for each row of matrix X."""

        # Subtract max for numerical stability
        e_x = np.exp(X - np.max(X, axis=1).reshape([-1, 1]))

        # We usually want diagonal probailities to be 0.
        if zero_index is None:
            if diag_zero:
                np.fill_diagonal(e_x, 0.)
        else:
            e_x[:, zero_index] = 0.

        # Add a tiny constant for stability of log we take later
        e_x = e_x + 1e-8  # numerical stability

        return e_x / e_x.sum(axis=1).reshape([-1, 1])


    def calc_prob_matrix(distances, sigmas=None, zero_index=None):
        """Convert a distances matrix to a matrix of probabilities."""
        if sigmas is not None:
            two_sig_sq = 2. * np.square(sigmas.reshape((-1, 1)))
            return softmax(distances / two_sig_sq, zero_index=zero_index)
        else:
            return softmax(distances, zero_index=zero_index)


    def binary_search(eval_fn, target, tol=1e-10, max_iter=10000,
                    lower=1e-20, upper=1000.):
        """Perform a binary search over input values to eval_fn.
        # Arguments
            eval_fn: Function that we are optimising over.
            target: Target value we want the function to output.
            tol: Float, once our guess is this close to target, stop.
            max_iter: Integer, maximum num. iterations to search for.
            lower: Float, lower bound of search range.
            upper: Float, upper bound of search range.
        # Returns:
            Float, best input value to function found during search.
        """
        for i in range(max_iter):
            guess = (lower + upper) / 2.
            val = eval_fn(guess)
            if val > target:
                upper = guess
            else:
                lower = guess
            if np.abs(val - target) <= tol:
                break
        return guess


    def calc_perplexity(prob_matrix):
        """Calculate the perplexity of each row
        of a matrix of probabilities."""
        entropy = -np.sum(prob_matrix * np.log2(prob_matrix), 1)
        perplexity = 2 ** entropy
        return perplexity


    def perplexity(distances, sigmas, zero_index):
        """Wrapper function for quick calculation of
        perplexity over a distance matrix."""
        return calc_perplexity(
            calc_prob_matrix(distances, sigmas, zero_index))


    def find_optimal_sigmas(distances, target_perplexity):
        """For each row of distances matrix, find sigma that results
        in target perplexity for that role."""
        sigmas = []
        # For each row of the matrix (each point in our dataset)
        for i in range(distances.shape[0]):
            # Make fn that returns perplexity of this row given sigma
            eval_fn = lambda sigma: \
                perplexity(distances[i:i+1, :], np.array(sigma), i)
            # Binary search over sigmas to achieve target perplexity
            correct_sigma = binary_search(eval_fn, target_perplexity)
            # Append the resulting sigma to our output array
            sigmas.append(correct_sigma)
        return np.array(sigmas)



    def q_tsne(Y):
        """t-SNE: Given low-dimensional representations Y, compute
        matrix of joint probabilities with entries q_ij."""
        distances = neg_squared_euc_dists(Y)
        inv_distances = np.power(1. - distances, -1)
        np.fill_diagonal(inv_distances, 0.)
        return inv_distances / np.sum(inv_distances), inv_distances


    def tsne_grad(P, Q, Y, distances):
        """t-SNE: Estimate the gradient of the cost with respect to Y."""
        pq_diff = P - Q  # NxN matrix
        pq_expanded = np.expand_dims(pq_diff, 2)  # NxNx1
        y_diffs = np.expand_dims(Y, 1) - np.expand_dims(Y, 0)  # NxNx2
        # Expand our distances matrix so can multiply by y_diffs
        distances_expanded = np.expand_dims(distances, 2)  # NxNx1
        # Weight this (NxNx2) by distances matrix (NxNx1)
        y_diffs_wt = y_diffs * distances_expanded  # NxNx2
        grad = 4. * (pq_expanded * y_diffs_wt).sum(1)  # Nx2
        return grad


    def p_joint(X, target_perplexity):
        """Given a data matrix X, gives joint probabilities matrix.
        # Arguments
            X: Input data matrix.
        # Returns:
            P: Matrix with entries p_ij = joint probabilities.
        """
        # Get the negative euclidian distances matrix for our data
        distances = neg_squared_euc_dists(X)
        # Find optimal sigma for each row of this distances matrix
        sigmas = find_optimal_sigmas(distances, target_perplexity)
        # Calculate the probabilities based on these optimal sigmas
        p_conditional = calc_prob_matrix(distances, sigmas)
        # Go from conditional to joint probabilities matrix
        P = p_conditional_to_joint(p_conditional)
        return P


    def estimate_sne(X, y, P, rng, num_iters, q_fn, grad_fn, learning_rate,
                    momentum, plot):
        """Estimates a SNE model.
        # Arguments
            X: Input data matrix.
            y: Class labels for that matrix.
            P: Matrix of joint probabilities.
            rng: np.random.RandomState().
            num_iters: Iterations to train for.
            q_fn: Function that takes Y and gives Q prob matrix.
            plot: How many times to plot during training.
        # Returns:
            Y: Matrix, low-dimensional representation of X.
        """

        # Initialise our 2D representation
        Y = rng.normal(0., 0.0001, [X.shape[0], 2])

        # Initialise past values (used for momentum)
        if momentum:
            Y_m2 = Y.copy()
            Y_m1 = Y.copy()

        # Start gradient descent loop
        for i in range(num_iters):

            # Get Q and distances (distances only used for t-SNE)
            Q, distances = q_fn(Y)
            # Estimate gradients with respect to Y
            grads = grad_fn(P, Q, Y, distances)

            # Update Y
            Y = Y - learning_rate * grads
            if momentum:  # Add momentum
                Y += momentum * (Y_m1 - Y_m2)
                # Update previous Y's for momentum
                Y_m2 = Y_m1.copy()
                Y_m1 = Y.copy()

            # Plot sometimes
            if plot and i % (num_iters / plot) == 0:
                categorical_scatter_2d(Y, y, alpha=1.0, ms=6,
                                    show=True, figsize=(9, 6))

        return Y

In [7]:
from unsupervised.SVD_np import SVD
from numpy import array

In [4]:
A = array([
 [1,2,3,4,5,6,7,8,9,10],
 [11,12,13,14,15,16,17,18,19,20],
 [21,22,23,24,25,26,27,28,29,30]])

In [8]:
my_svd = SVD(n_components = 2)
my_svd.fit(A)
y = my_svd.transform(A)

In [9]:
y

array([[-18.52157747,   6.47697214],
       [-49.81310011,   1.91182038],
       [-81.10462276,  -2.65333138]])

In [7]:
from unsupervised.svd import SVD

In [ ]:
my_svd = SVD(n_components = 1)
my_svd.fit(input_matrix)
my_svd.transform(input_matrix)